In [1]:
!nvidia-smi

/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
Sat Dec 18 08:20:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.142.00   Driver Version: 450.142.00   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   79C    P0    70W /  70W |   3817MiB / 15109MiB |    100%      Defaul

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import time
import tensorflow as tf
from itertools import product
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
from numpy import array
import logging

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

## 로깅 사용

In [4]:
# 로깅 초기화
logger = logging.getLogger()
logger.setLevel(logging.INFO)
formatter = logging.Formatter(u'%(asctime)s %(message)s')
# StreamHandler
streamingHandler = logging.StreamHandler()
streamingHandler.setFormatter(formatter)
logger.addHandler(streamingHandler)
# FileHandler
file_handler = logging.FileHandler('getLog.log')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

# 모델의 n_steps만큼의 데이터 넣기...

## 최고 성능(accuracy) 모델의 하이퍼파라미터 불러오기

In [28]:
# Load highest accuracy AI model info
high_acc_model = keras.models.load_model("high_CNN-LSTM_SP500.h5")
model_idx = int(high_acc_model.get_config()['name'].split('_')[1])

df_result = pd.read_csv('result.csv', sep=',')
df_hypParams_Info = df_result[model_idx:model_idx+1]
df_hypParams_Info
#pred_now = high_acc_model.predict(until_now)
#pred_now

,index,epochs,learning_rate,test_ratio,n_steps,scaler,batch,is_predict,tp,tn,fp,fn,accuracy,recall,precision,loss_history_1,loss_history_2,loss_history_3,loss_history_4,time
63,63,50,0.001,0.3,90,10,512,Y,425,2,306,2,0.58125,0.19486,0.116643,0.015513,0.015491,0.01532,0.014936,0:00:33


## 사용할 데이터 불러오기

In [71]:
df_raw = pd.read_csv('SP_SPX, 1D_until211218.csv'
                ,parse_dates=['time']
                ,index_col=['time'])
df_raw.index=pd.to_datetime(df_raw.index, unit='s')

# raw_data의 마지막 날짜 부터 n_steps 날짜까지의 데이터 구하기
df_close_untilNow = pd.DataFrame(df_raw['close'][-df_hypParams_Info['n_steps'].values[0]:])


# Model 에 들어갔던 데이터와 동일한 data 만들기 (로그차분, scaler 등)

df_processed_data = np.log(df_close_untilNow).diff()*df_hypParams_Info['scaler'].values[0]
df_processed_data = np.array(df_processed_data).reshape(( df_processed_data.shape[1],df_processed_data.shape[0], len(df_close_untilNow.columns)))
df_processed_data[:10]

array([[[            nan],
        [ 1.60603743e-02],
        [ 2.62190141e-02],
        [-7.09025024e-02],
        [-1.08043916e-01],
        [ 1.25595203e-02],
        [ 8.10857715e-02],
        [ 8.48769836e-02],
        [ 1.49457519e-02],
        [ 2.21989018e-02],
        [-5.84194983e-02],
        [ 8.76902602e-02],
        [ 4.29733196e-02],
        [-1.35005417e-02],
        [ 3.11712776e-03],
        [ 2.83411365e-02],
        [-3.35083708e-03],
        [-3.40127368e-02],
        [-1.31501773e-02],
        [-4.62067912e-02],
        [-7.75263494e-02],
        [ 2.27392733e-02],
        [-5.75868590e-02],
        [ 8.43372417e-02],
        [-1.54783407e-02],
        [-9.15717422e-02],
        [-1.71230593e-01],
        [-8.12681549e-03],
        [ 9.47456232e-02],
        [ 1.20662386e-01],
        [ 1.45545067e-02],
        [-2.78022378e-02],
        [-2.05697735e-01],
        [ 1.56334505e-02],
        [-1.19812486e-01],
        [ 1.14305689e-01],
        [-1.30709038e-01],
 

## 예측하기

In [76]:
pred_tomorrow = high_acc_model.predict(df_processed_data)

if pred_tomorrow>0 :
    print('상승  예측 diff :', pred_tomorrow)
else : print('하락  예측 diff :', pred_tomorrow)

상승  예측 diff : [[0.01108564]]


In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
train_df_with_pred['close'].plot(ax=ax, legend=True)
train_df_with_pred['pred'].plot(ax=ax, legend=True)
ax.set_title("train t:"+str(df_result['test_ratio'][i])+", "+\
              "n:"+str(int(df_result['n_steps'][i]))+", "+\
              "s:"+str(df_result['scaler'][i])+", "+\
              "b:"+str(int(df_result['batch'][i]))
            )